In [ ]:
env = MinesweeperEnv()
agent = Agent(env.height*env.width, env.height*env.width)

EPISODES = 100000

episodes, scores, score_avg, total_action, length_memory, win_list = [], [], [], [], [], []

for epi in range(EPISODES):
    done = False
    score = 0
    total_action_epi = []

    # 게임 다시 시작; 게임판 초기화
    state = env.reset()
    while not done:

        # 현재 상태를 기반으로 행동을 선택
        action = agent.get_action(state)
        total_action_epi.append(action)
        next_state, reward, exploded, done = env.step(action)
        score += reward
        agent.append_sample(state, action, next_state, reward, exploded, done)

        # memory의 길이가 최소 이상일 때 훈련
        # batch_size만큼의 데이터가 훈련
        if len(agent.memory) > agent.memory_size_min:
            agent.train_model()
            if done:
              agent.lr_scheduler.step()

        state = next_state

        if done:

          if not env.exploded:
              win_list.append(1)
          else:
              win_list.append(0)

          if (epi % agent.update_time) == 0:
              agent.update_target_model()

          total_action.append(len(total_action_epi))
          episodes.append(epi)
          scores.append(score)
          score_avg.append(np.mean(scores[-100:]))
          length_memory.append(len(agent.memory))

          if (epi % 100) == 0:
              win_rate = np.mean(win_list[-100:]) if len(win_list) >= 100 else np.mean(win_list) # -100이므로 마지막에서 100번째 최신 것을 반영함.
              loss_avg = np.mean(agent.loss_list[-100:]) if len(win_list) >= 100 else np.mean(win_list)
              action_avg = np.mean(total_action[-100:]) if len(total_action) >= 100 else np.mean(total_action)
              print(f"Episode: {epi} | Score: {np.median(scores[-100:]):.5f} | Epsilon: {agent.epsilon:.2f} | Win Rate: {win_rate}, Loss avg: {loss_avg:.5f} | action_list: {action_avg}")